In [1]:
from Extern import preproc_load_raw
from Extern import tmsi_poly5reader
from matplotlib import pyplot as plt
import scipy
from scipy.signal import spectrogram, hann, butter, filtfilt
from importlib import reload
import dat_preproc
import pandas as pd
import numpy as np
import os
import mne

In [2]:
dat_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial\\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107'

raw_file = os.path.join(
    dat_path,
    '572NR53_MedOn1_StimOff_Rest1-20230915T140107.DATA.Poly5'
)

In [3]:
raw = tmsi_poly5reader.Poly5Reader(raw_file)

Reading file  S:\AG\AG-Bewegungsstoerungen-II\LFP\PROJECTS\ENTRAINMENT\Extern_Trial\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107\572NR53_MedOn1_StimOff_Rest1-20230915T140107.DATA.Poly5
	 Number of samples:  1020952 
	 Number of channels:  30 
	 Sample rate: 4096 Hz
Done reading data.


In [4]:
raw = raw.read_data_MNE()

Creating RawArray with float64 data, n_channels=30, n_times=1020952
    Range : 0 ... 1020951 =      0.000 ...   249.256 secs
Ready.


t:\Dokumente\PROJECTS\DYSKINESIA_PROJECT\FTG_GithubCode\Extern\tmsi_poly5reader.py:94: RuntimeWarning: Channel names are not unique, found duplicates for: {'Z', 'X', 'Y'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=labels, sfreq=fs, ch_types=types_clean)


In [5]:
tstamps_sec = (1 / raw.info['sfreq']) * np.arange(raw.n_times)
new_ch_name = 'TimeinSec'
new_data_2d = tstamps_sec.reshape(1,-1)
new_info = mne.create_info(['Time_Sec'], raw.info['sfreq'], ch_types=['misc'])

# Create a RawArray object for the new channel
new_raw_array = mne.io.RawArray(new_data_2d, new_info)

# Add the new channel to the original Raw object
raw.add_channels([new_raw_array])

Creating RawArray with float64 data, n_channels=1, n_times=1020952
    Range : 0 ... 1020951 =      0.000 ...   249.256 secs
Ready.


<RawArray | 31 x 1020952 (249.3 s), ~241.5 MB, data loaded>

In [6]:
fif_name = os.path.join(dat_path, 'Sub001_Extern_MedOn_StimOff.fif')
raw.save(fif_name, overwrite = True)

Writing S:\AG\AG-Bewegungsstoerungen-II\LFP\PROJECTS\ENTRAINMENT\Extern_Trial\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107\Sub001_Extern_MedOn_StimOff.fif
    Writing channel names to FIF truncated to 15 characters with remapping


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_17760\2744524795.py:2: RuntimeWarning: This filename (S:\AG\AG-Bewegungsstoerungen-II\LFP\PROJECTS\ENTRAINMENT\Extern_Trial\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107\Sub001_Extern_MedOn_StimOff.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(fif_name, overwrite = True)


Closing S:\AG\AG-Bewegungsstoerungen-II\LFP\PROJECTS\ENTRAINMENT\Extern_Trial\572NR53_MedOn1_StimOff_Rest1 - 20230915T140107\Sub001_Extern_MedOn_StimOff.fif
[done]


In [ ]:
raw = mne.io.read_raw_fif(os.path.join(
    'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial',
        'Sub001_Extern_MedOn_StimOn_L2.fif'
    )
)

In [7]:
raw.ch_names

['LFPR01STN',
 'LFPR02STN',
 'LFPR03STN',
 'LFPR04STN',
 'LFPR05STN',
 'LFPR06STN',
 'LFPR07STN',
 'LFPR08STN',
 'LFPL01STN',
 'LFPL02STN',
 'LFPL03STN',
 'LFPL04STN',
 'LFPL05STN',
 'LFPL06STN',
 'LFPL07STN',
 'LFPL08STN',
 'EEGCz',
 'EEGFz',
 'BIP 01',
 'BIP 02',
 'BIP 03',
 'X-0',
 'Y-0',
 'Z-0',
 'X-1',
 'Y-1',
 'Z-1',
 'ISO aux',
 'STATUS',
 'Counter 2power24',
 'Time_Sec']

In [ ]:
# Calculate the notch frequencies for the harmonics
harmonics = [130, 260, 390, 520, 650, 780, 910, 1040, 1170, 1300, 1430, 1560, 1690, 1820, 1950]  # You can adjust the number of harmonics as needed

samp_freq  = raw.info['sfreq']  # Sample frequency (Hz)
notch_freq  = 130
quality_factor  = 30.0  # Quality factor
# Design notch filter
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, samp_freq)


In [8]:
# Filter the list to select only elements containing 'LFPL'
chans = [s for s in raw.ch_names if 'LFP' in s]
#chans = ['LFPL01STN', 'LFPL02STN', 'LFPL03STN', 'LFPL04STN', 'LFPL08STN']
print(chans)

['LFPR01STN', 'LFPR02STN', 'LFPR03STN', 'LFPR04STN', 'LFPR05STN', 'LFPR06STN', 'LFPR07STN', 'LFPR08STN', 'LFPL01STN', 'LFPL02STN', 'LFPL03STN', 'LFPL04STN', 'LFPL05STN', 'LFPL06STN', 'LFPL07STN', 'LFPL08STN']


In [18]:
%matplotlib qt
from scipy import signal
DETREND = 1
NOTCH = 1

for ch in chans:
    dat = raw.get_data(picks = ch)[0]
    fs = raw.info['sfreq']
    stimulation_frequencies = [260, 390]
    Q = 30  # Q factor for notch filter
    window_length = int(fs)  # 1-second window length
    overlap = window_length // 4  # 25% overlap

    # Detrend the LFP signal
    if DETREND == 1:
        dat = signal.detrend(dat)

    # Apply notch filters for each frequency and its harmonics
    if NOTCH == 1:
        filtered_signals = []
        for freq in stimulation_frequencies:
            w0 = freq / (fs / 2)  # Normalized notch frequency
            b, a = signal.iirnotch(w0, Q)
            filtered_signal = signal.filtfilt(b, a, dat)
            filtered_signals.append(filtered_signal)
            # Combine the filtered signals
            dat = np.sum(filtered_signals, axis=0)

    # Calculate the short-time Fourier transform (STFT) using Hamming window
    frequencies, times, Zxx = signal.stft(dat, fs=fs, nperseg=window_length, noverlap=overlap, window='hamming')

    # Plot the spectral amplitudes
    plt.figure()
    plt.pcolormesh(times, frequencies, 20 * np.log10(np.abs(Zxx)), shading='auto', cmap='inferno', vmin=-200, vmax = -60)
    plt.yticks(np.arange(0, 512, 30))
    plt.title(str(ch))
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.colorbar(label='Amplitude (dB)')
    plt.ylim(1, 512)  # Set the frequency range as desired
    plt.show()

    plt.savefig(os.path.join(dat_path, f'{ch}_StimOff_FFT'), dpi = 200)


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_17760\4143926698.py:28: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 20 * np.log10(np.abs(Zxx)), shading='auto', cmap='inferno', vmin=-200, vmax = -60)
